In [1]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

Qiskit version: 1.1.2


In [2]:
from qiskit import QuantumCircuit

# Create a quantum circuit with one qbit
qc = QuantumCircuit(1)

# Define initial_state as |1> = [0,1], and in plain English, it is a qubit resulting in the value 1 when observed.
initial_state = [0,1]

# Apply initialization operation on the qbit at position 0
qc.initialize(initial_state, 0)

In [10]:
# Prepare the simulation backend
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

# Create an AerSimulator with statevector
backend = AerSimulator()
#backend = Aer.get_backend('statevector_simulator')

# Transpile the circuit for backend
qc_transpiled = transpile(qc, backend)

# Run the circuit
job = backend.run(qc_transpiled, shots=1, memory=True, save_statevector=True)
result = job.result()


In [11]:
# The measured qbit
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Get the probability distribution
counts = result.get_counts()

# Show the histogram
plot_histogram(counts)

QiskitError: 'No counts for experiment "0"'